In [1]:
# 查看GPU信息
!nvidia-smi

Thu May 12 15:29:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000000:06:00.0 Off |                    0 |
| N/A   53C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 13320, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 13320 (delta 30), reused 38 (delta 29), pack-reused 13277
Receiving objects: 100% (13320/13320), 12.00 MiB | 42.00 KiB/s, done.
Resolving deltas: 100% (9278/9278), done.
/ipynb/yolov5


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="DXGaQFBdatLlmobAlwRW")
project = rf.workspace("graduation-project-workspace").project("data-zlgg9")
dataset = project.version(4).download("yolov5")

Looking in indexes: https://mirrors.sjtug.sjtu.edu.cn/pypi/web/simple
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to data-4 in yolov5pytorch:: 100%|██████████| 6122/6122 [00:01<00:00, 3445.88it/s]


In [3]:
# 查看数据集摘要（roboflow为我们生成的YAML文件）
%cat {dataset.location}/data.yaml

names:
- fallen_tree
- litter
- vegetation_loss
- well_loss
nc: 4
train: data-4/train/images
val: data-4/valid/images


In [4]:
# 从上面YAML文件中读取数据集中的类数
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [5]:
# 使用YOLOv5提供的yolov5s模型
# 查看一下模型结构
%cat models/yolov5s.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]]

In [6]:
# 使用@register_line_cell_magic自定义魔术命令模版供后续使用
from IPython.core.magic import register_line_cell_magic

# writetemplate函数作用：将全局变量写到文件中
@register_line_cell_magic
# line是魔术命令（一行内），cell对应ipynb中的一个单元格
def writetemplate(line, cell):
    with open(line, 'w') as f:
        # cell.format(**globals())是收集单元格内的内容
        f.write(cell.format(**globals()))

In [7]:
%%writetemplate models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [14]:
# 下载预训练权重
# 网络问题，建议直接上传
import wget

target_dir = "weights"
%mkdir {target_dir}
down_url = "https://github.com/ultralytics/yolov5/releases/download/v6.1/yolov5s.pt"
wget.download(down_url, target_dir) # 下载

mkdir: cannot create directory ‘weights’: File exists


RemoteDisconnected: Remote end closed connection without response

In [8]:
# 使用tensorboard查看训练效果
%load_ext tensorboard
%tensorboard --logdir runs/train
# 使用ssh隧道在local上查看ssh -L 6006:127.0.0.1:6006 -p 26849 root@hz-t3.matpool.com
# tensorboard默认是6006端口

In [9]:
%%time
# 使用命令行方式训练模型
# 参数如下：
# img: 定义输入图片size
# batch: batch_size
# epoch: 训练循环次数（3000+）
# data: yaml路径
# cfg: 具体配置
# weights: 预训练权重文件路径
# name: 结果名
# nosave: 只保存最终checkpoint
# cache: 缓存图片提高训练速度
!python train.py --img 416 --batch 16 --epochs 500 --data {dataset.location}/data.yaml --cfg ./models/custom_yolov5s.yaml --weights weights/yolov5s.pt --name yolov5s_results  --cache

train: weights=weights/yolov5s.pt, cfg=./models/custom_yolov5s.yaml, data=/ipynb/yolov5/data-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=500, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v6.1-178-g4870064 torch 1.11.0+cu102 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_